In [1]:
import numpy as np
import os
import torch
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

torch.manual_seed(10)

In [5]:
import multiprocessing as mp
max_cpus = mp.cpu_count()
batch_size = 64 #consider going up later

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
# Get the mean and sd values for each of the channel of the images to be able to use them for the actual transformation

transform = transforms.Compose([
    transforms.ToTensor(),  # Convert the image to a tensor with values between 0 and 1
])

# Load the dataset
my_dataset = datasets.ImageFolder(root="dataset/", transform=transform)

# Compute the mean and standard deviation of pixel values for each channel separately
mean = np.mean([np.mean(x.numpy(), axis=(1,2)) for x, _ in my_dataset], axis=0)
std = np.mean([np.std(x.numpy(), axis=(1,2)) for x, _ in my_dataset], axis=0)

In [35]:
def generate_embeddings():
    """
    Transform, resize and normalize the images and then use a pretrained model to extract 
    the embeddings.
    """
    # TODO: define a transform to pre-process the images
    train_transforms = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224), transforms.RandomHorizontalFlip() ,transforms.ToTensor(), transforms.Normalize(mean, std)])

    train_dataset = datasets.ImageFolder(root="dataset/", transform=train_transforms)
    # Hint: adjust batch_size and num_workers to your PC configuration, so that you don't 
    # run out of memory
    train_loader = DataLoader(dataset=train_dataset,
                              batch_size= batch_size,
                              shuffle=False,
                              pin_memory=True, num_workers=max_cpus)

    # TODO: define a model for extraction of the embeddings (Hint: load a pretrained model,
    #  more info here: https://pytorch.org/vision/stable/models.html)

    model = models.resnet50(pretrained = True)

    # # Remove last layer to obtain embeddings
    model_embeddings = nn.Sequential(*list(model.children())[:-1])
    model_embeddings = model_embeddings.to(device)

    embedding_size = 2048 # Dummy variable, replace with the actual embedding size once you 
    # pick your model
    num_images = len(train_dataset)
    embeddings = np.zeros((num_images, embedding_size))

    with torch.no_grad():
        for i, data in enumerate(train_loader):

            bacth_embeddings = model_embeddings(data[0])
            bacth_embeddings = bacth_embeddings.view(bacth_embeddings.size(0), -1)
            bacth_embeddings = bacth_embeddings.to(device)
            start_index = i * train_loader.batch_size
            end_index = start_index + bacth_embeddings.size(0)
            embeddings[start_index:end_index] = bacth_embeddings.cpu().numpy()

            del bacth_embeddings

    # # TODO: Use the model to extract the embeddings. Hint: remove the last layers of the 
    # # model to access the embeddings the model generates. 

    np.save('dataset/embeddings.npy', embeddings)

In [ ]:
# train_loader = generate_embeddings()
# train_features, train_labels = next(iter(train_loader))
# img = train_features[0].squeeze()
# label = train_labels[0]
# plt.imshow(img[0], cmap="gray")
# plt.show()

In [ ]:
# model = models.resnet18(pretrained = True)

# # Remove last layer to obtain embeddings
# model_embeddings = nn.Sequential(*list(model.children())[:-1])
# model_embeddings = model_embeddings.to(device)

# a = model_embeddings(train_features)
# aa = a.view(a.size(0), -1)

In [36]:
generate_embeddings()

c:\Users\schni\OneDrive\Documents\Switzerland\Bottmedical\.venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [37]:
embeddings = np.load('dataset/embeddings.npy')

In [6]:
def normalize_embeddings(embeddings):
    l2_norm = np.linalg.norm(embeddings, axis=1, keepdims=True)
    norm_emb = embeddings/l2_norm
    return norm_emb

In [7]:
def get_data(file, train=True):
    """
    Load the triplets from the file and generate the features and labels.

    input: file: string, the path to the file containing the triplets
          train: boolean, whether the data is for training or testing

    output: X: numpy array, the features
            y: numpy array, the labels
    """
    triplets = []
    with open(file) as f:
        for line in f:
            triplets.append(line)

    # generate training data from triplets
    train_dataset = datasets.ImageFolder(root="dataset/",
                                         transform=None)
    filenames = [s[0].split('/')[-1].replace('.jpg', '') for s in train_dataset.samples]
    embeddings = np.load('dataset/embeddings.npy')
    # TODO: Normalize the embeddings across the dataset
    embeddings = normalize_embeddings(embeddings)

    file_to_embedding = {}
    for i in range(len(filenames)):
        file_to_embedding[filenames[i].replace('food\\', '')] = embeddings[i]
    X = []
    y = []
    # use the individual embeddings to generate the features and labels for triplets
    for t in triplets:
        emb = [file_to_embedding[a] for a in t.split()]
        X.append(np.hstack([emb[0], emb[1], emb[2]]))
        y.append(1)
        # Generating negative samples (data augmentation)
        if train:
            X.append(np.hstack([emb[0], emb[2], emb[1]]))
            y.append(0)
    X = np.vstack(X)
    y = np.hstack(y)
    return X, y

In [9]:
# Hint: adjust batch_size and num_workers to your PC configuration, so that you don't run out of memory
def create_loader_from_np(X, y = None, train = True, batch_size = batch_size, shuffle=True, num_workers = max_cpus):
    """
    Create a torch.utils.data.DataLoader object from numpy arrays containing the data.

    input: X: numpy array, the features
           y: numpy array, the labels
    
    output: loader: torch.data.util.DataLoader, the object containing the data
    """
    if train:
        dataset = TensorDataset(torch.from_numpy(X).type(torch.float), 
                                torch.from_numpy(y).type(torch.long))
    else:
        dataset = TensorDataset(torch.from_numpy(X).type(torch.float))
    loader = DataLoader(dataset=dataset,
                        batch_size=batch_size,
                        shuffle=shuffle,
                        pin_memory=True, num_workers=num_workers)
    return loader

In [37]:
# TODO: define a model. Here, the basic structure is defined, but you need to fill in the details
class Net(nn.Module):
    """
    The model class, which defines our classifier.
    """
    def __init__(self):
        """
        The constructor of the model.
        """
        super().__init__()
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

        self.fc1 = nn.Linear(2048*3, 2048)
        self.fc2 = nn.Linear(2048, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 1)

    def forward(self, x):
        """
        The forward pass of the model.

        input: x: torch.Tensor, the input to the model

        output: x: torch.Tensor, the output of the model
        """
        x = self.fc1(x)
        x = self.relu(x)

        x = self.fc2(x)
        x = self.relu(x)

        x = self.fc3(x)
        x = self.relu(x)
        
        x = self.fc4(x)
        
        return x

In [44]:
def train_model(train_loader):
    """
    The training procedure of the model; it accepts the training data, defines the model 
    and then trains it.

    input: train_loader: torch.data.util.DataLoader, the object containing the training data
    
    output: model: torch.nn.Module, the trained model
    """
    model = Net()
    model.train()
    model.to(device)
    n_epochs = 20

    # TODO: define a loss function, optimizer and proceed with training. Hint: use the part 
    # of the training data as a validation split. After each epoch, compute the loss on the 
    # validation split and print it out. This enables you to see how your model is performing 
    # on the validation data before submitting the results on the server. After choosing the 
    # best model, train it on the whole training data.

    loss_function = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    X_train, y_train = train_loader.dataset.tensors

    X_train, X_val, y_train, y_val = train_test_split(X_train.numpy(), y_train.numpy(), test_size=0.20, random_state = 1)

    train_loader = create_loader_from_np(X_train, y_train, train=True, batch_size = batch_size)
    val_loader = create_loader_from_np(X_val, y_val, train=True, batch_size = batch_size)

    for epoch in range(n_epochs):        
        for [X, y] in train_loader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(X).squeeze()
            loss = loss_function(outputs, y.type(torch.float))
            loss.backward()
            optimizer.step()

        # Validation loop
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for [X_val, y_val] in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)
                outputs_val = model(X_val).squeeze()
                loss_val = loss_function(outputs_val, y_val.type(torch.float))
                val_loss += loss_val.item()

        val_loss /= len(val_loader)
        print(f"Epoch {epoch+1}/{n_epochs}, Validation Loss: {val_loss:.4f}")

        # Switch back to train mode
        model.train()
    return model

In [45]:
def test_model(model, loader):
    """
    The testing procedure of the model; it accepts the testing data and the trained model and 
    then tests the model on it.

    input: model: torch.nn.Module, the trained model
           loader: torch.data.util.DataLoader, the object containing the testing data
        
    output: None, the function saves the predictions to a results.txt file
    """
    model.eval()
    predictions = []
    # Iterate over the test data
    with torch.no_grad(): # We don't need to compute gradients for testing
        for [x_batch] in loader:
            x_batch= x_batch.to(device)
            predicted = model(x_batch)
            predicted = predicted.cpu().numpy()
            # Rounding the predictions to 0 or 1
            predicted[predicted >= 0.5] = 1
            predicted[predicted < 0.5] = 0
            predictions.append(predicted)
        predictions = np.vstack(predictions)
    np.savetxt("results_2.txt", predictions, fmt='%i')

In [46]:
TRAIN_TRIPLETS = 'train_triplets.txt'
TEST_TRIPLETS = 'test_triplets.txt'
X, y = get_data(TRAIN_TRIPLETS)
X_test, _ = get_data(TEST_TRIPLETS, train=False)

In [47]:
train_loader = create_loader_from_np(X, y, train = True, batch_size =  batch_size)
test_loader = create_loader_from_np(X_test, train = False, batch_size=2048, shuffle=False)

In [48]:
model = train_model(train_loader)

Epoch 1/20, Validation Loss: 129.1953
Epoch 2/20, Validation Loss: 128.4733
Epoch 3/20, Validation Loss: 128.1851
Epoch 4/20, Validation Loss: 127.7206
Epoch 5/20, Validation Loss: 127.8271
Epoch 6/20, Validation Loss: 126.7614
Epoch 7/20, Validation Loss: 127.0026
Epoch 8/20, Validation Loss: 126.5300
Epoch 9/20, Validation Loss: 125.8055
Epoch 10/20, Validation Loss: 125.8481
Epoch 11/20, Validation Loss: 125.2706
Epoch 12/20, Validation Loss: 125.9160
Epoch 13/20, Validation Loss: 125.0563
Epoch 14/20, Validation Loss: 126.0779
Epoch 15/20, Validation Loss: 125.3078
Epoch 16/20, Validation Loss: 130.0714
Epoch 17/20, Validation Loss: 130.5260
Epoch 18/20, Validation Loss: 134.4161
Epoch 19/20, Validation Loss: 126.5535
Epoch 20/20, Validation Loss: 127.5071


In [49]:
# test the model on the test data
test_model(model, test_loader)
print("Results saved to results.txt")

Results saved to results.txt


In [1]:
class Net(nn.Module):
    """
    The model class, which defines our classifier.
    """
    def __init__(self):
        """
        The constructor of the model.
        """
        super().__init__()
        
        self.relu = nn.ReLU()
        #self.dropout = nn.Dropout(0.5)

        self.fc1 = nn.Linear(2048*3, 3072)
        self.fc2 = nn.Linear(3072, 1536)
        self.fc3 = nn.Linear(1536, 768)
        self.fc4 = nn.Linear(768, 384)
        self.fc5 = nn.Linear(384, 192)
        self.fc6 = nn.Linear(192, 96)
        self.fc7 = nn.Linear(96, 48)
        self.fc8 = nn.Linear(48, 24)
        self.fc9 = nn.Linear(24, 12)
        self.fc10 = nn.Linear(12, 6)
        self.fc11 = nn.Linear(6, 3)
        self.fc11 = nn.Linear(3, 1)

    def forward(self, x):
        """
        The forward pass of the model.

        input: x: torch.Tensor, the input to the model

        output: x: torch.Tensor, the output of the model
        """
        x = self.fc1(x)
        x = self.relu(x)

        x = self.fc2(x)
        x = self.relu(x)

        x = self.fc3(x)
        x = self.relu(x)

        x = self.fc4(x)
        x = self.relu(x)

        x = self.fc5(x)
        x = self.relu(x)

        x = self.fc6(x)
        x = self.relu(x)

        x = self.fc7(x)
        x = self.relu(x)

        x = self.fc8(x)
        x = self.relu(x)

        x = self.fc9(x)
        x = self.relu(x)

        x = self.fc10(x)
        x = self.relu(x)
        
        x = self.fc11(x)
        
        return x
    
def train_model(train_loader):
    """
    The training procedure of the model; it accepts the training data, defines the model 
    and then trains it.

    input: train_loader: torch.data.util.DataLoader, the object containing the training data
    
    output: model: torch.nn.Module, the trained model
    """
    model = Net()
    model.train()
    model.to(device)
    n_epochs = 10

    # TODO: define a loss function, optimizer and proceed with training. Hint: use the part 
    # of the training data as a validation split. After each epoch, compute the loss on the 
    # validation split and print it out. This enables you to see how your model is performing 
    # on the validation data before submitting the results on the server. After choosing the 
    # best model, train it on the whole training data.

    loss_function = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    X_train, y_train = train_loader.dataset.tensors

    X_train, X_val, y_train, y_val = train_test_split(X_train.numpy(), y_train.numpy(), test_size=0.20, random_state = 1)

    train_loader = create_loader_from_np(X_train, y_train, train=True, batch_size = batch_size)
    val_loader = create_loader_from_np(X_val, y_val, train=True, batch_size = batch_size)

    for epoch in range(n_epochs):        
        for [X, y] in train_loader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(X).squeeze()
            loss = loss_function(outputs, y.type(torch.float))
            loss.backward()
            optimizer.step()

        # Validation loop
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for [X_val, y_val] in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)
                outputs_val = model(X_val).squeeze()
                loss_val = loss_function(outputs_val, y_val.type(torch.float))
                val_loss += loss_val.item()

        val_loss /= len(val_loader)
        print(f"Epoch {epoch+1}/{n_epochs}, Validation Loss: {val_loss:.4f}")

        # Switch back to train mode
        model.train()
    return model

def test_model(model, loader):
    """
    The testing procedure of the model; it accepts the testing data and the trained model and 
    then tests the model on it.

    input: model: torch.nn.Module, the trained model
           loader: torch.data.util.DataLoader, the object containing the testing data
        
    output: None, the function saves the predictions to a results.txt file
    """
    model.eval()
    predictions = []
    # Iterate over the test data
    with torch.no_grad(): # We don't need to compute gradients for testing
        for [x_batch] in loader:
            x_batch= x_batch.to(device)
            predicted = model(x_batch)
            predicted = predicted.cpu().numpy()
            # Rounding the predictions to 0 or 1
            predicted[predicted >= 0.5] = 1
            predicted[predicted < 0.5] = 0
            predictions.append(predicted)
        predictions = np.vstack(predictions)
    np.savetxt("results_4.txt", predictions, fmt='%i')

model = train_model(train_loader)
test_model(model, test_loader)
print("Results saved to results.txt")

NameError: name 'nn' is not defined

In [ ]:
# TODO: define a model. Here, the basic structure is defined, but you need to fill in the details
class Net(nn.Module):
    """
    The model class, which defines our classifier.
    """
    def __init__(self):
        """
        The constructor of the model.
        """
        super().__init__()
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

        self.fc1 = nn.Linear(2048*3, 2048)
        self.fc2 = nn.Linear(2048, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 1)

    def forward(self, x):
        """
        The forward pass of the model.

        input: x: torch.Tensor, the input to the model

        output: x: torch.Tensor, the output of the model
        """
        x = self.fc1(x)
        x = self.relu(x)

        x = self.fc2(x)
        x = self.relu(x)

        x = self.fc3(x)
        x = self.relu(x)
        
        x = self.fc4(x)
        
        return x
    

def train_model(train_loader):
    """
    The training procedure of the model; it accepts the training data, defines the model 
    and then trains it.

    input: train_loader: torch.data.util.DataLoader, the object containing the training data
    
    output: model: torch.nn.Module, the trained model
    """
    model = Net()
    model.train()
    model.to(device)
    n_epochs = 50

    # TODO: define a loss function, optimizer and proceed with training. Hint: use the part 
    # of the training data as a validation split. After each epoch, compute the loss on the 
    # validation split and print it out. This enables you to see how your model is performing 
    # on the validation data before submitting the results on the server. After choosing the 
    # best model, train it on the whole training data.

    loss_function = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    X_train, y_train = train_loader.dataset.tensors

    X_train, X_val, y_train, y_val = train_test_split(X_train.numpy(), y_train.numpy(), test_size=0.20, random_state = 1)

    train_loader = create_loader_from_np(X_train, y_train, train=True, batch_size = batch_size)
    val_loader = create_loader_from_np(X_val, y_val, train=True, batch_size = batch_size)

    for epoch in range(n_epochs):        
        for [X, y] in train_loader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(X).squeeze()
            loss = loss_function(outputs, y.type(torch.float))
            loss.backward()
            optimizer.step()

        # Validation loop
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for [X_val, y_val] in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)
                outputs_val = model(X_val).squeeze()
                loss_val = loss_function(outputs_val, y_val.type(torch.float))
                val_loss += loss_val.item()

        val_loss /= len(val_loader)
        print(f"Epoch {epoch+1}/{n_epochs}, Validation Loss: {val_loss:.4f}")

        # Switch back to train mode
        model.train()
    return model

def test_model(model, loader):
    """
    The testing procedure of the model; it accepts the testing data and the trained model and 
    then tests the model on it.

    input: model: torch.nn.Module, the trained model
           loader: torch.data.util.DataLoader, the object containing the testing data
        
    output: None, the function saves the predictions to a results.txt file
    """
    model.eval()
    predictions = []
    # Iterate over the test data
    with torch.no_grad(): # We don't need to compute gradients for testing
        for [x_batch] in loader:
            x_batch= x_batch.to(device)
            predicted = model(x_batch)
            predicted = predicted.cpu().numpy()
            # Rounding the predictions to 0 or 1
            predicted[predicted >= 0.5] = 1
            predicted[predicted < 0.5] = 0
            predictions.append(predicted)
        predictions = np.vstack(predictions)
    np.savetxt("results_3.txt", predictions, fmt='%i')

model = train_model(train_loader)
test_model(model, test_loader)
print("Results saved to results.txt")

In [ ]:
TRAIN_TRIPLETS = 'train_triplets.txt'
TEST_TRIPLETS = 'test_triplets.txt'

# generate embedding for each image in the dataset
# if(os.path.exists('dataset/embeddings.npy') == False):
#     generate_embeddings()

# load the training and testing data
X, y = get_data(TRAIN_TRIPLETS)
X_test, _ = get_data(TEST_TRIPLETS, train=False)

# Create data loaders for the training and testing data
train_loader = create_loader_from_np(X, y, train = True, batch_size=64)
test_loader = create_loader_from_np(X_test, train = False, batch_size=2048, shuffle=False)

# define a model and train it
model = train_model(train_loader)

# test the model on the test data
test_model(model, test_loader)
print("Results saved to results.txt")